# **Transfer Learning**

**Goal**: Using punlicly available image classification model and alter it to meet our needs, vastly improving both training speed and model performance.

**Objectives**

- Load data.
- Download a pre-trained model.
- Alter that model to suit our needs with Transfer Learning.
- Train just the altered part.
- Test model performance with K-fold cross-validation.

In [16]:
import os 

import matplotlib
import matplotlib.pyplot as plt 
import numpy as np 
import sklearn.model_selection
import torch
import torch.nn as nn 
import torch.optim as optim
import torchinfo
import torchvision
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from torch.utils.data import DataLoader
from torchinfo import summary
from torchvision import datasets, transforms
from tqdm import tqdm

In [17]:
print("torch version : ", torch.__version__)
print("torchvision version : ", torchvision.__version__)
print("torchinfo version : ", torchinfo.__version__)
print("numpy version : ", np.__version__)
print("matplotlib version : ", matplotlib.__version__)

!python --version

torch version :  2.5.1+cpu
torchvision version :  0.20.1+cpu
torchinfo version :  1.8.0
numpy version :  1.26.3
matplotlib version :  3.8.3
Python 3.12.1


In [18]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device.")

Using cpu device.


**1. Loading Data**

In [19]:
data_dir = os.path.join("data_p2", "data_undersampled", "train")

print("Data directory:", data_dir)

Data directory: data_p2\data_undersampled\train


Convert to RGB

In [20]:
class ConvertToRGB(object):
    def __call__(self, img):
        if img.mode != "RGB":
            img = img.convert("RGB")
        return img

Data Transformation Setup

In [21]:
transform_normalized = transforms.Compose(
    [
        ConvertToRGB(),
        transforms.Resize((224, 224)),
        # Convert images to tensors
        transforms.ToTensor(),
        transforms.Normalize(
            mean = [0.4326, 0.4953, 0.3120], std = [0.2178, 0.2214, 0.2091]
        )
        
    ]
)
transform_normalized

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.4326, 0.4953, 0.312], std=[0.2178, 0.2214, 0.2091])
)

ImageFolder Application

In [22]:
dataset = datasets.ImageFolder(data_dir, transform_normalized)

dataset

Dataset ImageFolder
    Number of datapoints: 8180
    Root location: data_p2\data_undersampled\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.4326, 0.4953, 0.312], std=[0.2178, 0.2214, 0.2091])
           )

In [23]:
classes = dataset.classes
classes

['cassava-bacterial-blight-cbb',
 'cassava-brown-streak-disease-cbsd',
 'cassava-green-mottle-cgm',
 'cassava-healthy',
 'cassava-mosaic-disease-cmd']

Class Balance Clarifiaction

In [24]:
from training import class_counts

In [ ]:
counts = class_counts(dataset)
counts

  0%|          | 0/8180 [00:00<?, ?it/s]

DataLoader Creation

In [ ]:
batch_size = 32
dataset_loader = DataLoader(dataset, batch_size)

print(f"Batch shape: {next(iter(dataset_loader))[0].shape}")

**2. Implementing Transfer Learning**